In [ ]:
import utils.utils as utils
example_aggregated = "./aggregated_results/cfg_single_lon120.obj"
aggregated_results = utils.load_results(example_aggregated)


# Create Neural Networks

In [ ]:
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense

def dense_nn(input_shape, output_shape, hidden_layers, activation):
    """
    Creates a dense NN in base of the parameters received
    """
    model = Sequential()
    model.add(Input(shape = input_shape))
    
    for n_layer_nodes in hidden_layers:
        model.add(Dense(n_layer_nodes, activation = activation))
    
    model.add(Dense(output_shape))
    return model
    
    
def fc_model(input_shape, output_shape, hidden_layers, activation, conservation_layer=False,
             inp_sub=None, inp_div=None, norm_q=None):
    inp = Input(shape=(input_shape,))

    # First hidden layer
    x = Dense(hidden_layers[0])(inp)
    x = act_layer(activation)(x)

    # Remaining hidden layers
    for h in hidden_layers[1:]:
        x = Dense(h)(x)
        x = act_layer(activation)(x)

    if conservation_layer:
        x = SurRadLayer(inp_sub, inp_div, norm_q)([inp, x])
        x = MassConsLayer(inp_sub, inp_div, norm_q)([inp, x])
        out = EntConsLayer(inp_sub, inp_div, norm_q)([inp, x])

    else:
        out = Dense(output_shape)(x)

    return tf.keras.models.Model(inp, out)


In [ ]:
class ModelDescription:

    def __init__(self, variable, pc_alpha, threshold, parents):
        self.variable = variable
        self.pc_alpha = pc_alpha
        self.threshold = threshold
        self.parents = parents
        self.model = self.build_model()
    
    def build_model(self):
        input_shape = len(parents)
        input_shape = (input_shape, )
        model = dense_nn(
            input_shape = input_shape,
            output_shape = 1, # Only one variable
            hidden_layers = [32, 32, 32],
            activation = "relu",
        )
        return model
    
    def __str__(self):
        return f"{self.variable}: a{self.pc_alpha}-t{self.threshold}"
    
    def __repr__(self):
        return repr(str(self))
    
    def __hash__(self):
        return hash(str(self))
    
    def __eq__(self, other):
        return str(self) == str(other)


model_descriptions = list()
for variable, pc_alpha_dict in aggregated_results.items():
    print(variable)
    for pc_alpha, pc_alpha_results in pc_alpha_dict.items():
        for threshold, parents in pc_alpha_results["parents"].items():
            model_description = ModelDescription(
                variable, pc_alpha, threshold, parents
            )
            model_descriptions.append(model_description)


# Training

In [ ]:
for model_description in model_descriptions:
    print(model_description)
    
